# NMDS Gradients

In class, we performed NMDS by initializing points for each object (i.e., animal) randomly and then iteratively moving points in a direction that will reduce stress. In this assignment, we will implement the calculation of these directions along with the rest of the NMDS algorithm.

You will need to refer to the lecture notebooks from class as well as what you learned in discussion section to complete this assignment.

In [ ]:
!pip install -q otter-grader

import otter
grader = otter.Notebook("hw6.ipynb")

import numpy as np
import pandas as pd

The human similarity data is loaded below. We want to find a set of points that is related to these similarity ratings in the in the way that Shepard's Law predicts. That set of points is an inferred psychological representation.

In [ ]:
sim_vals = [
 [1., 0.600554459474065, 0.7536383164437648, 0.5312856091329679],
 [0.600554459474065, 1., 0.49306869139523984, 0.7288934141100247],
 [0.7536383164437648, 0.49306869139523984, 1., 0.4088417197978041],
 [0.5312856091329679, 0.7288934141100247, 0.4088417197978041, 1.]
]
labels = ['dog', 'cat', 'wolf', 'rabbit']
df_sim = pd.DataFrame(sim_vals, columns=labels, index=labels)
df_sim

First we will randomly initialize points for each animal:

In [ ]:
# do not change
np.random.seed(10)

# do not change
def create_initial_random_points():
    points = np.random.rand(4, 2)
    points_df = pd.DataFrame(
        points, 
        columns=['dim1', 'dim2'], 
        index=['dog', 'cat', 'wolf', 'rabbit']
    )
    return points_df

# do not change
df_guesses = create_initial_random_points()
df_guesses

As discussed in section, NMDS iteratively adjusts each point $x_i$ with respect to each other point $x_j$ (where $i \neq j$) using the formula:

$x_i = x_i + \text{step\_size} * (d̂ᵢⱼ - dᵢⱼ) × (x_j - x_i)/d̂ᵢⱼ$,

where $dᵢⱼ$ is distance in psychological space, $d̂ᵢⱼ$ is the distance between points we are adjusting, and $(d̂ᵢⱼ - dᵢⱼ) × (x_j - x_i)/d̂ᵢⱼ$ is called the **gradient**.

To review only briefly, $(x_j - x_i)/d̂ᵢⱼ$ is a unit vector that points from $x_i$ to $x_j$, and $(d̂ᵢⱼ - dᵢⱼ)$ is the important signed term in the stress that determines (1) whether we step in the direction of $(x_j - x_i)/d̂ᵢⱼ$ or $-(x_j - x_i)/d̂ᵢⱼ$, and (2) the size of the step we take relative to other points.

**Exercise 1:**

Perform NMDS using the following criteria:

- Store gradients for all points on all iterations in a multidimensional numpy array called `directions` with shape `(n_iterations, n_animals, n_animals - 1, 2)`.
- Set $\text{step\_size}$ to $0.4$.
- Run for $100$ iterations.
- Store the stress for each iteration in an array called `stress_vals`.

In [ ]:
# Your code goes here


# DO NOT CHANGE
for stress_val in stress_vals:
    print(stress_val)

In [ ]:
grader.check("q1")